# Time Series Forecasting Models
- In this task, we build a time series forecasting model to predict Tesla's (TSLA) future stock prices. 
- We explore and compare different modeling techniques, including ARIMA, SARIMA, and LSTM. 
- The goal is to develop a robust model that can accurately forecast stock prices and provide actionable insights for portfolio management.

In [1]:
import os, sys

notebook_dir = os.getcwd()

#add path
sys.path.append(os.path.abspath(os.path.join(notebook_dir, '..')))

In [2]:
from scripts.fetchPrepr_close_column import *
from scripts.sarima_model import *
from scripts.lstm_model import *

In [ ]:
# import pandas as pd
# Fetch and preprocess data
tsla_data = fetch_data('TSLA', '2015-01-01', '2025-01-31')
train_data, test_data = preprocess_data(tsla_data)

# index = pd.date_range(start='2023-01-24', periods=len(data['Close']), freq='D')
# test_data = pd.DataFrame(test_data, index=index)

# Extract the 'Close' column as a Series
# test_data_series = test_data['Close']

[*********************100%***********************]  1 of 1 completed


In [ ]:
print(train_data)
print(test_data)
# test_data.shape

Ticker            TSLA
Date                  
2023-01-24  143.889999
2023-01-25  144.429993
2023-01-26  160.270004
2023-01-27  177.899994
2023-01-30  166.660004
...                ...
2025-01-24  406.579987
2025-01-27  397.149994
2025-01-28  398.089996
2025-01-29  389.100006
2025-01-30  400.279999

[507 rows x 1 columns]


In [10]:
# SARIMA Model
print("Training SARIMA model...")
sarima_model = fit_sarima(train_data)
sarima_predictions, sarima_mae, sarima_rmse, sarima_mape = evaluate_sarima(sarima_model, test_data)
print(f"SARIMA Metrics - MAE: {sarima_mae}, RMSE: {sarima_rmse}, MAPE: {sarima_mape}")

Training SARIMA model...
Performing stepwise search to minimize aic


c:\Users\pc\Desktop\10_Academy\Week-11\kaim-week-11\.myvenv11\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\pc\Desktop\10_Academy\Week-11\kaim-week-11\.myvenv11\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\pc\Desktop\10_Academy\Week-11\kaim-week-11\.myvenv11\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\pc\Desktop\10_Academy\Week-11\kaim-week-11\.myvenv11\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\pc\Desktop\10_Academy\Week-11\kaim-week-11\.myvenv1

 ARIMA(2,1,2)(1,0,1)[12] intercept   : AIC=12678.187, Time=4.34 sec
 ARIMA(0,1,0)(0,0,0)[12] intercept   : AIC=12680.726, Time=0.05 sec


c:\Users\pc\Desktop\10_Academy\Week-11\kaim-week-11\.myvenv11\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\pc\Desktop\10_Academy\Week-11\kaim-week-11\.myvenv11\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


 ARIMA(1,1,0)(1,0,0)[12] intercept   : AIC=12680.415, Time=0.55 sec


...

 ARIMA(0,1,1)(2,0,0)[12]             : AIC=12647.203, Time=1.93 sec


c:\Users\pc\Desktop\10_Academy\Week-11\kaim-week-11\.myvenv11\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


 ARIMA(2,1,1)(2,0,0)[12]             : AIC=12649.632, Time=6.32 sec

Best model:  ARIMA(1,1,0)(2,0,0)[12]          
Total fit time: 199.267 seconds
Best SARIMA parameters: (1, 1, 0), (2, 0, 0, 12)


c:\Users\pc\Desktop\10_Academy\Week-11\kaim-week-11\.myvenv11\lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\pc\Desktop\10_Academy\Week-11\kaim-week-11\.myvenv11\lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
c:\Users\pc\Desktop\10_Academy\Week-11\kaim-week-11\.myvenv11\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


ValueError: Data must be 1-dimensional, got ndarray of shape (507, 507) instead